In [ ]:
# qlogin -q short.qg -l gpu=1[affinity=true],gputype=rtx8000
# # qlogin -q short.qg -l gpu=1[affinity=true],gputype=a100

srun -p gpu_short --gres gpu:1 --pty bash

In [ ]:
#!/bin/bash

cd /well/win/users/hsv459/SwinAgeMapper

module purge
module load Python/3.7.4-GCCcore-8.3.0

CPU_ARCHITECTURE=$(/apps/misc/utils/bin/get-cpu-software-architecture.py)

# Error handling
if [[ ! $? == 0 ]]; then
  echo "Fatal error: Please send the following information to the BMRC team: Could not determine CPU software architecture on $(hostname)"
  exit 1
fi

# Activate the ivybridge or skylake version of your python virtual environment
source /well/win/users/hsv459/python/swinagemapper-${CPU_ARCHITECTURE}/bin/activate
# source /well/win/users/hsv459/python/functionmapper-skylake/bin/activate

# continue to use your python venv as normal
   
# ipython



In [ ]:
%reset

In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.utils.data as data
import numpy as np

from solver import Solver
from utils.data_utils import get_datasets_dynamically, get_test_datasets_dynamically
from utils.settings import Settings
import utils.data_evaluation as evaluations

from SwinAgeMapper import SwinAgeMapper


# Set the default floating point tensor type to FloatTensor
torch.set_default_tensor_type(torch.FloatTensor)

settings_file_name = 'settings.ini'
evaluation_settings_file_name = 'settings_evaluation.ini'

settings = Settings(settings_file_name)
data_parameters = settings['DATA']
training_parameters = settings['TRAINING']
network_parameters = settings['NETWORK']
misc_parameters = settings['MISC']

########################################################################################
network_parameters['feature_size'] = 12
# network_parameters['depths'] = (1, 1, 1, 1)
network_parameters['num_heads'] = (2, 4, 8, 16)

batch = 5
training_parameters['training_batch_size'] = batch
training_parameters['validation_batch_size'] = batch
########################################################################################


def load_data_dynamically(data_parameters, mapping_evaluation_parameters=None, flag='train'):
    
    if flag=='train':
        print("Data is loading...")
        train_data, validation_data, resolution = get_datasets_dynamically(data_parameters)
        print("Data has loaded!")
        print("Training dataset size is {}".format(len(train_data)))
        print("Validation dataset size is {}".format(len(validation_data)))
        return train_data, validation_data, resolution
    elif flag=='test':
        print("Data is loading...")
        test_data, volumes_to_be_used, prediction_output_statistics_name, resolution = get_test_datasets_dynamically(data_parameters, mapping_evaluation_parameters)
        print("Data has loaded!")
        len_test_data = len(test_data)
        print("Testing dataset size is {}".format(len_test_data))
        return test_data, volumes_to_be_used, prediction_output_statistics_name, len_test_data, resolution
    else:
        print('ERROR: Invalid Flag')
        return None
    
train_data, validation_data, resolution = load_data_dynamically(data_parameters=data_parameters, flag='train')

if training_parameters['optimiser'] == 'adamw':
    optimizer = torch.optim.AdamW
elif training_parameters['optimiser'] == 'adam':
    optimizer = torch.optim.Adam
else:
    optimizer = torch.optim.AdamW # Default option

optimizer_arguments={'lr': training_parameters['learning_rate'],
                    'betas': training_parameters['optimizer_beta'],
                    'eps': training_parameters['optimizer_epsilon'],
                    'weight_decay': training_parameters['optimizer_weigth_decay']
                    }

if training_parameters['loss_function'] == 'mse':
    loss_function = torch.nn.MSELoss()
elif training_parameters['loss_function'] == 'mae':
    loss_function = torch.nn.L1Loss()
else:
    print("Loss function not valid. Defaulting to MSE!")
    loss_function = torch.nn.MSELoss()
    
train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=training_parameters['training_batch_size'],
    shuffle=True,
    pin_memory=True,
    num_workers=data_parameters['num_workers']
)
validation_loader = data.DataLoader(
    dataset=validation_data,
    batch_size=training_parameters['validation_batch_size'],
    shuffle=False,
    pin_memory=True,
    num_workers=data_parameters['num_workers']
)

AgeMapperModel = SwinAgeMapper(
                            img_size = network_parameters['img_size'],
                            in_channels = network_parameters['in_channels'],
                            depths = network_parameters['depths'],
                            num_heads = network_parameters['num_heads'],
                            feature_size = network_parameters['feature_size'],
                            drop_rate = network_parameters['drop_rate'],
                            attn_drop_rate = network_parameters['attn_drop_rate'],
                            dropout_path_rate = network_parameters['dropout_path_rate'],
                            use_checkpoint = network_parameters['use_checkpoint'],
                            spatial_dims = network_parameters['spatial_dims'],
                            downsample = network_parameters['downsample'],
                            fully_connected_activation = network_parameters['fully_connected_activation'],
                            resolution=resolution,
                            )

solver = Solver(model=AgeMapperModel,
                number_of_classes=network_parameters['number_of_classes'],
                experiment_name=training_parameters['experiment_name'],
                optimizer=optimizer,
                optimizer_arguments=optimizer_arguments,
                loss_function=loss_function,
                model_name=training_parameters['experiment_name'],
                number_epochs=training_parameters['number_of_epochs'],
                loss_log_period=training_parameters['loss_log_period'],
                learning_rate_scheduler_gamma=training_parameters['learning_rate_scheduler_gamma'],
                use_last_checkpoint=training_parameters['use_last_checkpoint'],
                experiment_directory=misc_parameters['experiments_directory'],
                logs_directory=misc_parameters['logs_directory'],
                checkpoint_directory=misc_parameters['checkpoint_directory'],
                best_checkpoint_directory=misc_parameters['best_checkpoint_directory'],
                save_model_directory=misc_parameters['save_model_directory'],
                learning_rate_scheduler_flag = training_parameters['learning_rate_scheduler_flag'],
                learning_rate_scheduler_patience=training_parameters['learning_rate_scheduler_patience'],
                learning_rate_scheduler_threshold=training_parameters['learning_rate_scheduler_threshold'],
                learning_rate_scheduler_min_value=training_parameters['learning_rate_scheduler_min_value'],
                lr_cosine_scheduler_warmup_epochs = training_parameters['lr_cosine_scheduler_warmup_epochs'],
                early_stopping_patience=training_parameters['early_stopping_patience'],
                early_stopping_min_patience=training_parameters['early_stopping_min_patience'],
                early_stopping_min_delta=training_parameters['early_stopping_min_delta'],
                use_pre_trained = training_parameters['use_pre_trained'],
                )

solver.train(train_loader, validation_loader)

In [ ]:
[NETWORK]
img_size = (160, 192, 160)
patch_size=5
in_channels = 1
depths = (2, 2, 2, 2)
num_heads = (3, 6, 12, 24)
feature_size = 48
drop_rate = 0.0
attn_drop_rate = 0.0
dropout_path_rate = 0.0
use_checkpoint = True
spatial_dims = 3
downsample="merging"
fully_connected_activation="relu"
number_of_classes = 1

[TRAINING]
; ---> Model Properties
experiment_name = "SM0-55"


In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.utils.data as data
import numpy as np

from solver import Solver
from utils.data_utils import get_datasets_dynamically, get_test_datasets_dynamically
from utils.settings import Settings
import utils.data_evaluation as evaluations

from SwinAgeMapper import SwinAgeMapper


# Set the default floating point tensor type to FloatTensor
torch.set_default_tensor_type(torch.FloatTensor)

name = 'SM0-55'


settings_file_name = name + '.ini'
evaluation_settings_file_name = name + '_eval.ini'

settings = Settings(settings_file_name)
data_parameters = settings['DATA']
training_parameters = settings['TRAINING']
network_parameters = settings['NETWORK']
misc_parameters = settings['MISC']


# network_parameters['feature_size'] = 12
# network_parameters['depths'] = (1, 1, 1, 1)

batch = 3
training_parameters['training_batch_size'] = batch
training_parameters['validation_batch_size'] = batch

In [ ]:
def load_data_dynamically(data_parameters, mapping_evaluation_parameters=None, flag='train'):
    
    if flag=='train':
        print("Data is loading...")
        train_data, validation_data, resolution = get_datasets_dynamically(data_parameters)
        print("Data has loaded!")
        print("Training dataset size is {}".format(len(train_data)))
        print("Validation dataset size is {}".format(len(validation_data)))
        return train_data, validation_data, resolution
    elif flag=='test':
        print("Data is loading...")
        test_data, volumes_to_be_used, prediction_output_statistics_name, resolution = get_test_datasets_dynamically(data_parameters, mapping_evaluation_parameters)
        print("Data has loaded!")
        len_test_data = len(test_data)
        print("Testing dataset size is {}".format(len_test_data))
        return test_data, volumes_to_be_used, prediction_output_statistics_name, len_test_data, resolution
    else:
        print('ERROR: Invalid Flag')
        return None
    
train_data, validation_data, resolution = load_data_dynamically(data_parameters=data_parameters, flag='train')

if training_parameters['optimiser'] == 'adamw':
    optimizer = torch.optim.AdamW
elif training_parameters['optimiser'] == 'adam':
    optimizer = torch.optim.Adam
else:
    optimizer = torch.optim.AdamW # Default option

optimizer_arguments={'lr': training_parameters['learning_rate'],
                    'betas': training_parameters['optimizer_beta'],
                    'eps': training_parameters['optimizer_epsilon'],
                    'weight_decay': training_parameters['optimizer_weigth_decay']
                    }

if training_parameters['loss_function'] == 'mse':
    loss_function = torch.nn.MSELoss()
elif training_parameters['loss_function'] == 'mae':
    loss_function = torch.nn.L1Loss()
else:
    print("Loss function not valid. Defaulting to MSE!")
    loss_function = torch.nn.MSELoss()
    
train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=training_parameters['training_batch_size'],
    shuffle=True,
    pin_memory=True,
    num_workers=data_parameters['num_workers']
)
validation_loader = data.DataLoader(
    dataset=validation_data,
    batch_size=training_parameters['validation_batch_size'],
    shuffle=False,
    pin_memory=True,
    num_workers=data_parameters['num_workers']
)

AgeMapperModel = SwinAgeMapper(
                            img_size = network_parameters['img_size'],
                            in_channels = network_parameters['in_channels'],
                            depths = network_parameters['depths'],
                            num_heads = network_parameters['num_heads'],
                            feature_size = network_parameters['feature_size'],
                            drop_rate = network_parameters['drop_rate'],
                            attn_drop_rate = network_parameters['attn_drop_rate'],
                            dropout_path_rate = network_parameters['dropout_path_rate'],
                            use_checkpoint = network_parameters['use_checkpoint'],
                            spatial_dims = network_parameters['spatial_dims'],
                            downsample = network_parameters['downsample'],
                            fully_connected_activation = network_parameters['fully_connected_activation'],
                            resolution=resolution,
                            )

solver = Solver(model=AgeMapperModel,
                number_of_classes=network_parameters['number_of_classes'],
                experiment_name=training_parameters['experiment_name'],
                optimizer=optimizer,
                optimizer_arguments=optimizer_arguments,
                loss_function=loss_function,
                model_name=training_parameters['experiment_name'],
                number_epochs=training_parameters['number_of_epochs'],
                loss_log_period=training_parameters['loss_log_period'],
                learning_rate_scheduler_gamma=training_parameters['learning_rate_scheduler_gamma'],
                use_last_checkpoint=training_parameters['use_last_checkpoint'],
                experiment_directory=misc_parameters['experiments_directory'],
                logs_directory=misc_parameters['logs_directory'],
                checkpoint_directory=misc_parameters['checkpoint_directory'],
                best_checkpoint_directory=misc_parameters['best_checkpoint_directory'],
                save_model_directory=misc_parameters['save_model_directory'],
                learning_rate_scheduler_flag = training_parameters['learning_rate_scheduler_flag'],
                learning_rate_scheduler_patience=training_parameters['learning_rate_scheduler_patience'],
                learning_rate_scheduler_threshold=training_parameters['learning_rate_scheduler_threshold'],
                learning_rate_scheduler_min_value=training_parameters['learning_rate_scheduler_min_value'],
                lr_cosine_scheduler_warmup_epochs = training_parameters['lr_cosine_scheduler_warmup_epochs'],
                early_stopping_patience=training_parameters['early_stopping_patience'],
                early_stopping_min_patience=training_parameters['early_stopping_min_patience'],
                early_stopping_min_delta=training_parameters['early_stopping_min_delta'],
                use_pre_trained = training_parameters['use_pre_trained'],
                )

solver.train(train_loader, validation_loader)

In [ ]:
import os
import shutil
import argparse
import logging

import torch
import torch.utils.data as data
import numpy as np

from solver import Solver
from utils.data_utils import get_datasets_dynamically, get_test_datasets_dynamically
from utils.settings import Settings
import utils.data_evaluation as evaluations


# Set the default floating point tensor type to FloatTensor
torch.set_default_tensor_type(torch.FloatTensor)

def load_data_dynamically(data_parameters, mapping_evaluation_parameters=None, flag='train'):
    
    if flag=='train':
        print("Data is loading...")
        train_data, validation_data = get_datasets_dynamically(data_parameters)
        print("Data has loaded!")
        print("Training dataset size is {}".format(len(train_data)))
        print("Validation dataset size is {}".format(len(validation_data)))
        return train_data, validation_data
    elif flag=='test':
        print("Data is loading...")
        test_data, volumes_to_be_used, prediction_output_statistics_name = get_test_datasets_dynamically(data_parameters, mapping_evaluation_parameters)
        print("Data has loaded!")
        len_test_data = len(test_data)
        print("Testing dataset size is {}".format(len_test_data))
        return test_data, volumes_to_be_used, prediction_output_statistics_name, len_test_data
    else:
        print('ERROR: Invalid Flag')
        return None
    
settings_file_name = 'MMx-1.ini'

settings = Settings(settings_file_name)
data_parameters = settings['DATA']
training_parameters = settings['TRAINING']
network_parameters = settings['NETWORK']
misc_parameters = settings['MISC']

train_data, validation_data = load_data_dynamically(data_parameters=data_parameters, flag='train')

train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=1
)

i=0
for inputs, targets in train_loader:
    if i>1:
        break
    print(type(inputs), type(targets))
    print('----------')
    i+=1
        
print(inputs[0][:,80,90,50:55])
print(inputs[1][:,80,90,50:55])
print(inputs[2][:,80,90,50:55])
# print(np.divide(inputs[0][:,80,90,50:55], inputs[1][:,80,90,50:55]))

# tensor([[0.6915, 0.5228, 0.5321, 0.5967, 0.6413]])
# tensor([[0.4916, 0.6317, 0.6725, 0.6307, 0.5606]])
# tensor([[1.4067, 0.8276, 0.7912, 0.9461, 1.1440]])
# tensor([[1.4067, 0.8276, 0.7912, 0.9461, 1.1440]])